# Quick Tour

## Pipeline

pipeline()は，事前学習済みのモデルを推論に使用するための最も簡単で高速な方法です．
pipline()を使用することで，様々なモダリティ（テキスト，画像，音声）の多くのタスクに対してすぐに利用することができます．

[Hugging Face models](https://huggingface.co/models)にアクセスすることで，タスクを確認することができます．

まず，pipeline()のインスタンスを作成し，使用したいタスクを指定します．
以下の例では，センチメント分析と呼ばれる，文章が肯定的（positive）か否定的（negative）かを判定するタスクを扱います．

In [1]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/fujie/.conda/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pipeline()は，センチメント分析のためのデフォルトの事前学習済みモデルと，トークナイザを自動的にダウンロードしてキャッシュします．
2回目以降の呼び出しはキャッシュからモデルを読み込むため，ダウンロードが不要になります．

では，早速 classifier を使ってみましょう．

In [2]:
classifier("we are very happy to show you the 🤗 Transformers library.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

結果の `label` は感情のラベルを表し，`score` はその感情に対する確信度を表します．

結果はリストとして与えられます．
複数の入力が与えられた場合は，複数の結果がリストとして返されます．

In [4]:
classifier(["I am happy", "I am sad"])

[{'label': 'POSITIVE', 'score': 0.9998801946640015},
 {'label': 'NEGATIVE', 'score': 0.9991856217384338}]

次に自動音声認識を実行してみましょう．

まず `pipeline()` を使って speech_recognizer を作成します．
タスクは `automatic-speech-ecognition` とし，モデルに `facebook/wav2vec2-base-960h` を指定します．

In [6]:
speech_recognizer = pipeline("automatic-speech-recognition",
                             model="facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

次に音声データセットをロードします．

Hugging Face datasets については別途，チュートリアルなどを参照してください．

ここでは，MinDS-14 データセットをロードします．

In [10]:
from datasets import load_dataset, Audio
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

`dataset`の中の音声データのサンプリング周波数は 8kHz になっています．
`speech_recognizer` は 16kHz を想定しているため，サンプリング周波数の変換をします．

In [19]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

speech_recognizer に音声データを入力し，結果を表示します．

In [22]:
speech_recognizer(dataset[:5]["audio"])

[{'text': 'I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT'},
 {'text': "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE"},
 {'text': "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS"},
 {'text': 'HOW DO I FURN A JOINA COUT'},
 {'text': 'CAN NOW YOU HELP ME SET UP AN JOINT LEAKACCOUNT'}]

## pipelineで他のモデルとトークナイザを利用する

pipeline()は，[Hub](https://huggingface.co/models)から任意のモデルをダウンロードして利用することもできます．

例えば，日本語のモデルを利用してみます．

In [24]:
model_name = "abhishek/autonlp-japanese-sentiment-59363"

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [25]:
classifier("みなさんに会えて嬉しいです")

[{'label': 'positive', 'score': 0.9995540976524353}]

## AutoClass

`AutoClass`は，事前学習済みモデルのアーキテクチャをその名前やパスから自動的に推測し，適切なモデルを読み込むための便利なクラスです．

適切な`AutoClass`を選択し，それに関する前処理クラスを選択するだけで済みます．

### AutoTokenizer

トークナイザはテキストをモデルの入力として使用できる数値の配列に変換するためのものです．
トークナイズ処理は，単語をどのように分割するかなど，多くのルールがあります．
事前学習済みのモデルを使用する場合，そのモデルに対応するトークナイザを使用する必要があります．
そのため，同じモデル名でトークナイザをインスタンス化する必要があります．

In [26]:
from transformers import AutoTokenizer

model_name = "abhishek/autonlp-japanese-sentiment-59363"
tokenizer = AutoTokenizer.from_pretrained(model_name)

トークナイザにテキストを渡してみましょう:

In [27]:
encoding = tokenizer("みなさんに会えて嬉しいです")
print(encoding)

{'input_ids': [2, 12989, 11689, 893, 1136, 6426, 888, 1945, 11265, 12461, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


ここで
- `input_ids`: トークン ID （整数値）のリスト
- `attention_mask`: どのトークンにアテンションを向けるかを示すマスク

トークナイザは入力のリストを与えて，一様な長さのバッチを作成することができます．
このとき，必要に応じてテキストをパディングしたり，トリミングしたりします．

In [31]:
pt_batch = tokenizer(
    ["みなさんに会えて嬉しいです", "別れるのが辛いです"],
    padding=True,        # 最も長いシーケンスに合わせてパディング（0詰め）
    truncation=True,     # 最大トークン数を超える場合は切り捨て
    max_length=512,      # 最大トークン数
    return_tensors="pt", # PyTorchのテンソルを返す
)

In [32]:
print(pt_batch)

{'input_ids': tensor([[    2, 12989, 11689,   893,  1136,  6426,   888,  1945, 11265, 12461,
             3],
        [    2, 16170,  6365,   896,   862, 29301, 12461,     3,     0,     0,
             0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}


### AutoModel

`AutoModel`は，事前学習済みモデルのアーキテクチャをその名前やパスから自動的に推測し，適切なモデルを読み込むための便利なクラスです．
`AutoTokenizer`と同様に，適切な`AutoModel`を選択し，それに関する前処理クラスを選択するだけで済みます． 

In [34]:
from transformers import AutoModelForSequenceClassification

model_name = "abhishek/autonlp-japanese-sentiment-59363"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

前処理済みのバッチをモデルに渡してみましょう:

In [35]:
# **pt_batchは，辞書であるpt_batchを展開して引数として渡します．
pt_outputs = pt_model(**pt_batch)

In [36]:
pt_outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-3.8021,  3.9129],
        [-2.0204,  2.5722]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`pt_outputs`の`logits`は，softmax関数を適用することで確率に変換できます．

In [37]:
from torch import nn
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

tensor([[4.4587e-04, 9.9955e-01],
        [1.0024e-02, 9.8998e-01]], grad_fn=<SoftmaxBackward0>)


### モデルの保存

モデルをファインチューニングしたら，PreTrainedModel.save_pretrained()を使ってモデルを保存できます．

In [39]:
pt_save_directory = "./pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
pt_model.save_pretrained(pt_save_directory)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256}


保存したモデルは，PreTrainedModel.from_pretrained()を使って読み込むことができます．

In [40]:
pt_model = AutoModelForSequenceClassification.from_pretrained(pt_save_directory)

Hugging Face Transformers のモデルは，PyTorch，TensorFlow，JAXで保存されたモデルを相互に変換することができます．
（割愛）

## カスタムモデルの作成

モデル設定クラスを，事前学習モデルで初期化することで，カスタムモデルを作成することができます．

In [43]:
from transformers import AutoConfig, AutoModel

# 事前学習のモデルに対応する設定を読み込む
model_name = "distilbert/distilbert-base-uncased"
my_config = AutoConfig.from_pretrained(model_name, n_heads=12)

# 読み込んだ設定でモデルを初期化
my_model = AutoModel.from_config(my_config)

/home/fujie/.conda/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Trainer - PyTorch向けの最適化された学習ループ

全てのモデルは標準の `torch.nn.Module` であり，通常のループで学習することができます．

Hugging Face Transformers では，PyTorch向けに`Trainer`クラスを提供していて，基本的な学習ループに加えて，分散学習，混合精度などの機能が利用できます．

タスクに応じて，通常は`Trainer`に以下のパラメータを渡します:

1. `PreTrainedModel`または`torch.nn.Module`

In [46]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2. `TrainingArguments`には，変更できるモデルのハイパーパラメータが含まれています．
例えば，学習率，バッチサイズ，学習するエポック数などが変更できます．指定しない場合は，デフォルト値が使用されます:

In [47]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./saved_model", # モデルの保存先
    learning_rate=2e-5,         # 学習率
    per_device_train_batch_size=8, # バッチサイズ
    per_device_eval_batch_size=8,  # バッチサイズ
    num_train_epochs=3,         # エポック数
)
    

3. トークナイザ，画像処理，特徴抽出器などの前処理クラスをロードします:

In [48]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

4. データセットをロードします:

In [49]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes") 

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

5. データセットをトークナイズするための関数を作成します:

In [50]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

`map`を利用してデータセット全体に適用します．

In [51]:
dataset = dataset.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

6. データセットからの例のバッチを作成するための`DataCollatorWithPadding`を作成します:

In [52]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

次に，これらのクラスをTrainerにまとめます.

In [53]:
from transformers import Trainer

trainer = Trainer(
    model=model,                 # モデル
    args=training_args,          # 学習設定
    train_dataset=dataset["train"], # 訓練データ
    eval_dataset=dataset["test"],   # 評価データ
    tokenizer=tokenizer,          # トークナイザ
    data_collator=data_collator,  # データコレータ
)

最後に`train()`を呼び出して学習を開始します．

In [54]:
trainer.train()

Step,Training Loss
500,0.451700
1000,0.395200
1500,0.270300
2000,0.279900
2500,0.175200
3000,0.153200


TrainOutput(global_step=3201, training_loss=0.2779122042454842, metrics={'train_runtime': 109.8821, 'train_samples_per_second': 232.886, 'train_steps_per_second': 29.131, 'total_flos': 293653315380000.0, 'train_loss': 0.2779122042454842, 'epoch': 3.0})

`Trainer`をサブクラス化することで，学習ループをカスタマイズすることができます．
これにより，損失関数，オプティマイザ，スケジューラなどの機能をカスタマイズできます．

他にも，`Callbacks`を使用することもできます．
コールバックを利用して他のライブラリと連携し，学習ループを監視して進捗状況を報告したり，学習を早期に停止したりすることができます．
